In [ ]:
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import ImageFile
from tensorflow.keras.preprocessing import image as image_utils
import numpy as np
import cv2
from PIL import Image

# Setting the Env for gpu 

In [ ]:
import tensorflow as tf
from tensorflow.python.keras import backend as K

# adjust values to your needs
config = tf.compat.v1.ConfigProto( device_count = {'GPU': 0 } )
sess = tf.compat.v1.Session(config=config) 
K.set_session(sess)

# Inception V3 model

In [ ]:
base_model = keras.applications.InceptionV3(
    weights='imagenet', #<-change
    input_shape=(224, 224, 3),
    include_top=False)#<-Change

In [ ]:
base_model.trainable = False #<-change

In [ ]:
# Create inputs with correct shape
inputs = keras.Input(shape=(224, 224, 3))#<-Changed

x = base_model(inputs, training=False)

# Add pooling layer or flatten layer
x = keras.layers.GlobalAveragePooling2D()(x)#<-Changed

# Add final dense layer
outputs = keras.layers.Dense(5, activation = 'softmax')(x)

# Combine inputs and outputs to create model
model = keras.Model(inputs, outputs) #<-Change

In [ ]:
model.summary()

In [ ]:
model.compile(loss=keras.losses.BinaryCrossentropy(from_logits=True), metrics=[keras.metrics.BinaryAccuracy()])

In [ ]:
datagen = ImageDataGenerator(
        samplewise_center=True,  # set each sample mean to 0
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False) # we don't expect Bo to be upside-down so we will not flip vertically

In [ ]:
# load and iterate training dataset
train_it = datagen.flow_from_directory('FaceShape Dataset/training_set', 
                                       target_size=(224,224), 
                                       color_mode='rgb', 
                                       class_mode="categorical")
# load and iterate validation dataset
valid_it = datagen.flow_from_directory('FaceShape Dataset/testing_set', 
                                      target_size=(224,224), 
                                      color_mode='rgb', 
                                      class_mode="categorical")

In [ ]:
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
# Unfreeze the base model
base_model.trainable = True

# It's important to recompile your model after you make any changes
# to the `trainable` attribute of any inner layer, so that your changes
# are taken into account
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate = .00001),  # Very low learning rate
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
eff_history = model.fit_generator(train_it, validation_data = valid_it, steps_per_epoch = train_it.samples/train_it.batch_size, epochs = 15)
#45

In [ ]:
eff_history.model.evaluate(valid_it, steps=valid_it.samples/valid_it.batch_size)

# Binary

In [ ]:
#model.compile(optimizer = keras.optimizers.RMSprop(lr=0.00001), loss = keras.losses.BinaryCrossentropy(from_logits=False), metrics = [keras.metrics.BinaryAccuracy()])

In [ ]:
#eff_history = model.fit(train_it, validation_data = valid_it, steps_per_epoch = train_it.samples/train_it.batch_size, epochs = 10)


In [ ]:
#eff_history.model.evaluate(valid_it, steps=valid_it.samples/valid_it.batch_size)

# Inception v 3 methode 2


In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
base_model = InceptionV3(input_shape = (224, 224, 3), include_top = False, weights = 'imagenet')

In [ ]:
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
from tensorflow.keras.optimizers import RMSprop

x = keras.layers.Flatten()(base_model.output)
x = keras.layers.Dense(1024, activation='relu')(x)
x = keras.layers.Dropout(0.2)(x)

# Add a final sigmoid layer with 1 node for classification output
x = keras.layers.Dense(5, activation='softmax')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer = RMSprop(learning_rate=0.0001), loss = 'binary_crossentropy', metrics = ['acc'])

In [ ]:
# Unfreeze the base model
base_model.trainable = True

# It's important to recompile your model after you make any changes
# to the `trainable` attribute of any inner layer, so that your changes
# are taken into account
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate = .00001),  # Very low learning rate
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
inc_history = model.fit_generator(train_it, validation_data = valid_it, steps_per_epoch = train_it.samples/train_it.batch_size, epochs = 10)


# testing

In [ ]:
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import ImageFile
from tensorflow.keras.preprocessing import image as image_utils
import numpy as np
import cv2
from PIL import Image

In [ ]:
alphabet = ('heart','oblong','oval','round','square')
dictionary = {}
for i in range(5):
    dictionary[i] = alphabet[i]
dictionary

In [ ]:

def load_and_scale_image(image_path):
    image = image_utils.load_img(image_path,target_size=(224,224))
    return image

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
def show_image(image_path):
    image = mpimg.imread(image_path)
    plt.imshow(image)

In [ ]:
def predict_shape(file_path):
    #show_image(file_path)
    #image = load_and_scale_image(file_path)
    #image = image_utils.img_to_array(image)
    image = image_utils.img_to_array(file_path)
    image = image.reshape(1,224,224,3) 
    #image = image/255
    prediction = x.predict(image)
    proba=x.predict_on_batch(image)
    # convert prediction to letter
    predicted_shape = dictionary[np.argmax(prediction)]
    return predicted_shape,proba[0][np.argmax(prediction)]*100

In [ ]:
import mediapipe as mp

In [ ]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic
mp_drawing.DrawingSpec(color=(0,0,255), thickness=2, circle_radius=2)

In [ ]:
holistic=mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) 

In [ ]:

cap = cv2.VideoCapture(0)
while cap.isOpened(): 
    
    ret, frame = cap.read()
    image = cv2.resize(frame,(224,224),3)
    image2 = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    face_class,proba = predict_shape(image)
    #print(face_class, face_prob)
    print(face_class,round(proba,1))
    gray_img=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    #detect
    results = holistic.process(image2)
    image2 = cv2.cvtColor(image2, cv2.COLOR_RGB2BGR)
    # print(results.face_landmarks)
    # fce_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        # Recolor image back to BGR for rendering
    # 1. Draw face landmarks
    mp_drawing.draw_landmarks(image2, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                                mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                )
    #draw
    
            
    # Display Class
    cv2.putText(image2, 'CLASS'
                , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
    cv2.putText(image2, face_class.split(' ')[0]
                        , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    # Display Probability
    cv2.putText(image2, 'PROB'
                , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
    cv2.putText(image2, str(round(proba,1))
                   , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    
    
    #image = cv2.resize(image,(860,640),3)
    cv2.imshow('face shape ',  image2)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break

In [ ]:
cap.release()
cv2.destroyAllWindows()

In [ ]:
eff_history.model.save('face shape best')#model 2

In [ ]:
x = keras.models.load_model('face shape best') # load model best

In [ ]:

cap = cv2.VideoCapture(0)
while cap.isOpened(): 
    
    ret, frame = cap.read()
    image = cv2.resize(frame,(224,224),3)
    image2 = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    face_class,proba = predict_shape(image)
    #print(face_class, face_prob)
    print(face_class,round(proba,1))
    gray_img=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    #detect
    results = holistic.process(image2)
    image2 = cv2.cvtColor(image2, cv2.COLOR_RGB2BGR)
    # print(results.face_landmarks)
    # fce_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        # Recolor image back to BGR for rendering
    # 1. Draw face landmarks
    mp_drawing.draw_landmarks(image2, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                                mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                )
    #draw
    
            
    # Display Class
    cv2.putText(image2, 'CLASS'
                , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
    cv2.putText(image2, face_class.split(' ')[0]
                        , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    # Display Probability
    cv2.putText(image2, 'PROB'
                , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
    cv2.putText(image2, str(round(proba,1))
                   , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    
    
    #image = cv2.resize(image,(860,640),3)
    cv2.imshow('face shape ',  image2)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break